In [1]:
import torch
import torch.nn as nn
import opt_einsum

In [2]:
params = {}
hparams = {}
hparams['N_layers'] = 1
hparams['N'] = 16
hparams['k'] = 16
params['hparams'] = hparams

In [4]:
n = 10000
n**2 - (n-1)

99990001

In [6]:
class KandK(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.hparams = params['hparams']
        self.init_layers()
        
    def init_layers(self):
        self.layers1 = nn.ModuleList()
        #Layers
        for i in range(self.hparams['N_c']):
            self.layers1.append(nn.Linear(in_features=int(self.hparams['N']), out_features=int(self.hparams['k']), bias=True))
            self.layers1.append(nn.Linear(in_features=int(self.hparams['k']), out_features=int(self.hparams['k']), bias=False)) 


    def forward(self, x):
        F = x
        F_i = x
        lnabsTrF_i = torch.zeros((x.shape[0],self.hparams['N']))#, dtype=self.hparams['dtype'], device=self.device)
        TrF_i = opt_einsum.contract('nii->n', F_i)
        lnabsTrF_i[:,0] = torch.log(torch.abs(TrF_i))
        for i in range(1,self.hparams['N']):
            F_i = torch.matmul(F,F_i)
            TrF_i = opt_einsum.contract('nii->n', F_i)
            lnabsTrF_i[:,i] = torch.log(torch.abs(TrF_i))
        c = torch.zeros(x.shape[0],self.hparams['k'])#, dtype=self.hparams['dtype'], device=self.device)
        for i in range(0, len(self.layers1), 2):
            c_i = torch.exp(self.layers1[i](lnabsTrF_i))
            c_i = self.layers1[i+1](c_i)
            c += c_i
        return c

In [7]:
x = torch.rand(1,hparams['N'],hparams['N'])

In [8]:
model = KandK(params)
model

KandK(
  (layers1): ModuleList(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=False)
  )
  (layers2): ModuleList(
    (0): Linear(in_features=16, out_features=100, bias=False)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=False)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=100, bias=False)
    (5): ReLU()
    (6): Linear(in_features=100, out_features=100, bias=False)
    (7): ReLU()
    (8): Linear(in_features=100, out_features=16, bias=False)
  )
)

In [9]:
model(x)

tensor([[ 809308.3125, -282580.1250,   53416.7109,   -2056.7617, -105692.1562,
           83103.6406,  438222.8125, -596302.1250, -372350.1562, -414877.4062,
          966563.8750,   15350.7451,  558314.5625,  378550.8125, -489255.7500,
         -300074.0938]], grad_fn=<MmBackward0>)

In [10]:
sum(p.numel() for p in model.parameters())


33728

In [11]:
x = torch.rand(100,hparams['N'],hparams['N'])

In [14]:
torch.norm(x, dim=(-1,-2), p='fro')

torch.Size([100])